In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName("test") \
.getOrCreate()

In [2]:
df = spark.read.csv("/usr/local/Cellar/apache-spark/3.2.0/libexec/examples/src/main/resources/people.csv", header=True, sep=';')

In [3]:
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [4]:
df.count()

2

In [5]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)



In [6]:
df.select("name").show()

+-----+
| name|
+-----+
|Jorge|
|  Bob|
+-----+



In [7]:
spark = SparkSession\
.builder\
.appName("CTR")\
.getOrCreate()

In [8]:
from pyspark.sql.types import StructField, StringType, StructType, IntegerType

In [9]:
schema=StructType([
StructField("id",StringType(),True),
StructField("click",IntegerType(),True),
StructField("hour",IntegerType(),True),
StructField("C1",StringType(),True),
StructField("banner_pos",StringType(),True),
StructField("site_id",StringType(),True),
StructField("site_domain",StringType(),True),
StructField("site_category",StringType(),True),
StructField("app_id",StringType(),True),
StructField("app_domain",StringType(),True),
StructField("app_category",StringType(),True),
StructField("device_id",StringType(),True),
StructField("device_ip",StringType(),True),
StructField("device_model",StringType(),True),
StructField("device_type",StringType(),True),
StructField("device_conn_type",StringType(),True),
StructField("C14",StringType(),True),
StructField("C15",StringType(),True),
StructField("C16",StringType(),True),
StructField("C17",StringType(),True),
StructField("C18",StringType(),True),
StructField("C19",StringType(),True),
StructField("C20",StringType(),True),
StructField("C21",StringType(),True),])

In [10]:
df = spark.read.csv("file:/Users/hamidafzal/Documents/شخصی/etucation/arshad ut/1/machin_learning/homework6/train.csv", schema=schema,header=True)


In [11]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: string (nullable = true)
 |-- banner_pos: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- device_conn_type: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string (nullable = true)
 |-- C16: string (nullable = true)
 |-- C17: string (nullable = true)
 |-- C18: string (nullable = true)
 |-- C19: string (nullable = true)
 |-- C20: string (nullable = true)
 |-- C21: string (nullable = true)



In [12]:
df.count()

40428967

In [13]:
 df =df.drop('id').drop('hour').drop('device_id').drop('device_ip')

In [14]:
df = df.withColumnRenamed("click", "label")

In [15]:
df.columns

['label',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21']

In [16]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

In [17]:
df_train.cache()

DataFrame[label: int, C1: string, banner_pos: string, site_id: string, site_domain: string, site_category: string, app_id: string, app_domain: string, app_category: string, device_model: string, device_type: string, device_conn_type: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string]

In [18]:
df_train.count()

28303473

In [19]:
categorical = df_train.columns
categorical.remove('label')
print(categorical)

['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


In [20]:
from pyspark.ml.feature import StringIndexer
indexers = [
    StringIndexer(inputCol=c, outputCol=
                  "{0}_indexed".format(c)).setHandleInvalid("keep")
    for c in categorical]

In [21]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers])

In [22]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler( inputCols=encoder.getOutputCols(), outputCol="features")

In [23]:
stages = indexers + [encoder, assembler]
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)

In [24]:
one_hot_encoder = pipeline.fit(df_train)
df_train_encoded = one_hot_encoder.transform(df_train)
df_train_encoded.show()

+-----+----+----------+--------+-----------+-------------+--------+----------+------------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+----------+------------------+---------------+-------------------+---------------------+--------------+------------------+--------------------+--------------------+-------------------+------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------------+--------------------------+-----------------------+---------------------------+-----------------------------+----------------------+--------------------------+----------------------------+----------------------------+---------------------------+--------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+
|label|  C1|banner_pos| site_id|site_domain|s

In [25]:
df_train_encoded = df_train_encoded.select(["label", "features"])
df_train_encoded.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(31532,[5,7,3758,...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,1322,...|
|    0|(31532,[5,7,1322,...|
|    0|(31532,[5,7,1539,...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
+-----+--------------------+
only showing top 20 rows



In [26]:
df_train_encoded.cache()

DataFrame[label: int, features: vector]

In [27]:
df_train.unpersist()

DataFrame[label: int, C1: string, banner_pos: string, site_id: string, site_domain: string, site_category: string, app_id: string, app_domain: string, app_category: string, device_model: string, device_type: string, device_conn_type: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string]

In [28]:
df_test_encoded = one_hot_encoder.transform(df_test)
df_test_encoded = df_test_encoded.select(["label", "features"])
df_test_encoded.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,792,4...|
|    0|(31532,[5,7,1322,...|
|    0|(31532,[5,7,1322,...|
|    0|(31532,[5,7,4557,...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
|    0|(31532,[5,7,14,45...|
+-----+--------------------+
only showing top 20 rows



In [29]:
df_test_encoded.cache()
df_test.unpersist()

DataFrame[label: int, C1: string, banner_pos: string, site_id: string, site_domain: string, site_category: string, app_id: string, app_domain: string, app_category: string, device_model: string, device_type: string, device_conn_type: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string]

In [30]:
from pyspark.ml.classification import LogisticRegression
classifier = LogisticRegression(maxIter=20, regParam=0.001,elasticNetParam=0.001)

In [31]:
 lr_model = classifier.fit(df_train_encoded)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/hamidafzal/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-ac58a0a5bf87>", line 1, in <module>
    lr_model = classifier.fit(df_train_encoded)
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/pyspark/ml/base.py", line 161, in fit
    return self._fit(dataset)
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/pyspark/ml/wrapper.py", line 335, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/pyspark/ml/wrapper.py", line 332, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1309, in __call__
    return_value = get_return_value(
  File "/usr/local/Cellar/apache-spark/3

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
predictions = lr_model.transform(df_test_encoded)

In [ ]:
predictions.cache()
 predictions.show()